<a href="https://colab.research.google.com/github/harnalashok/hadoop/blob/main/Learning_spark_DaraFrames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 23rd March 2023
# Myfolder: github/hadoop
# Objectives:
#             i)  Install pyspark on colab
#             ii) Install koalas on colab
#                 (version installed is spark-3.3.2)
#
# Java 8 install: https://stackoverflow.com/a/58191107
# Hadoop install: https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/SingleCluster.html
# Spark install:  https://stackoverflow.com/a/64183749
#                 https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/

# Spark Reference API
a. [Quickstart](https://spark.apache.org/docs/latest/quick-start.html) <br>
b. Dataframe [APIs list](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html) at a glance<br>
c. Pandas [API](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/index.html) on spark <br>
c. ALso look at useful [this source code](https://spark.apache.org/docs/latest/api/python/_modules/pyspark/sql/functions.html) of functions that has examples<br>



# A. Full spark install
Installs `pyspark (spark-3.3.2-bin-hadoop3)` and `koalas`.<br>
Takes less than a minute to install

### 1.0 Libraries

In [66]:
# 1.0 How to set environment variable
import os  
import time  

## 2.0 Define some functions

#### ssh_install()

In [67]:
# 2.0 Function to install ssh client and sshd (Server)
def ssh_install():
  print("\n--1. Download and install ssh server----\n")
  ! sudo apt-get remove openssh-client openssh-server
  ! sudo apt install openssh-client openssh-server
  
  print("\n--2. Restart ssh server----\n")
  ! service ssh restart

#### Java install

In [68]:
# 3.0 Function to download and install java 8
def install_java():
  ! rm -rf /usr/java

  print("\n--Download and install Java 8----\n")
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null        # install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     # set environment variable

  !update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
  !update-alternatives --set javac /usr/lib/jvm/java-8-openjdk-amd64/bin/javac
  
  !mkdir -p /usr/java
  ! ln -s "/usr/lib/jvm/java-8-openjdk-amd64"  "/usr/java"
  ! mv "/usr/java/java-8-openjdk-amd64"  "/usr/java/latest"
  
  !java -version       #check java version
  !javac -version

#### setup ssh passphrase

In [69]:
# 6.0 Function tp setup ssh passphrase
def set_keys():
  print("\n---22. Generate SSH keys----\n")
  ! cd ~ ; pwd 
  ! cd ~ ; ssh-keygen -t rsa -P '' -f ~/.ssh/id_rsa
  ! cd ~ ; cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
  ! cd ~ ; chmod 0600 ~/.ssh/authorized_keys


#### Set environment

In [70]:
# 7.0 Function to set up environmental variables
def set_env():
  print("\n---23. Set Environment variables----\n")
  # 'export' command does not work in colab
  # https://stackoverflow.com/a/57240319
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre"   
  

#### function to install prerequisites
java and ssh<br>


In [71]:
# 8.0 Function to call all functions
def install_components():
  print("\n--Install java----\n")
  ssh_install()
  install_java()  
  #set_keys()
  set_env()


## 3.0 Install components
Start downloading, install and configure. Takes around 2 minutes<br>
Your <u>input *'y'* is required </u>at one place while overwriting earlier ssh keys

In [72]:
# 9.0 Start installation
start = time.time()
install_components()
end = time.time()
print("\n---Time taken----\n")
print((end- start)/60)


--Install java----


--1. Download and install ssh server----

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libboost-atomic-dev libboost-atomic1.71-dev libboost-atomic1.71.0
  libboost-chrono-dev libboost-chrono1.71-dev libboost-chrono1.71.0
  libboost-container-dev libboost-container1.71-dev libboost-container1.71.0
  libboost-context-dev libboost-context1.71-dev libboost-context1.71.0
  libboost-coroutine-dev libboost-coroutine1.71-dev libboost-coroutine1.71.0
  libboost-date-time-dev libboost-date-time1.71-dev libboost-date-time1.71.0
  libboost-exception-dev libboost-exception1.71-dev libboost-fiber-dev
  libboost-fiber1.71-dev libboost-fiber1.71.0 libboost-filesystem-dev
  libboost-filesystem1.71-dev libboost-filesystem1.71.0 libboost-graph-dev
  libboost-graph-parallel-dev libboost-graph-parallel1.71-dev
  libboost-graph-parallel1.71.0 libboost-gra

## 4.0 Install spark
koalas will also be installed

### Define functions

`findspark`: PySpark isn't on `sys.path` by default, but that doesn't mean it can't be used as a regular library. You can address this by either symlinking pyspark into your site-packages, or adding `pyspark` to `sys.path` at runtime. `findspark` does the latter.

In [73]:
# 1.0 Function to download and unzip spark
def spark_koalas_install():
  print("\n--1.1 Install findspark----\n")
  !pip install -q findspark

  print("\n--1.2 Install databricks Koalas----\n")
  !pip install koalas
  
  # This download link NEEDS TO BE CHECKED AGAIN
  print("\n--1.3 Download Apache tar.gz----\n")
            
  ! wget -c https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz 

  print("\n--1.4 Transfer downloaded content and unzip tar.gz----\n")
  !  mv /content/spark*   /opt/
  ! tar -xzf /opt/spark-3.3.2-bin-hadoop3.tgz  --directory /opt/

  print("\n--1.5 Check folder for files----\n")
  ! ls -la /opt


In [74]:
# 1.1 Function to set environment
def set_spark_env():
  print("\n---2. Set Environment variables----\n")
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
  os.environ["JRE_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre" 
  os.environ["SPARK_HOME"] = "/opt/spark-3.3.2-bin-hadoop3" 
  os.environ["SPARK_CONF_DIR"] = "/opt/spark-3.3.2-bin-hadoop3/conf"     
  os.environ["LD_LIBRARY_PATH"] += ":/opt/spark-3.3.2-bin-hadoop3/lib/native"
  os.environ["PATH"] += ":/opt/spark-3.3.2-bin-hadoop3/bin:/opt/spark-3.3.2-bin-hadoop3/sbin"
  print("\n---2.1. Check Environment variables----\n")
  # Check
  ! echo $PATH
  ! echo $LD_LIBRARY_PATH

In [75]:
# 1.2 Function to configure spark 
def spark_conf():
  print("\n---3. Configure spark to access hadoop----\n")
  !mv /opt/spark-3.3.2-bin-hadoop3/conf/spark-env.sh.template  /opt/spark-3.3.2-bin-hadoop3/conf/spark-env.sh
  #!echo "HADOOP_CONF_DIR=/opt/hadoop-3.2.2/etc/hadoop/" >> /opt/spark-3.1.2-bin-hadoop3.2/conf/spark-env.sh
  print("\n---3.1 Check ----\n")
  #!cat /opt/spark-3.1.1-bin-hadoop3.2/conf/spark-env.sh

### Install spark


In [76]:
# 2.0 Call all the three functions
def install_spark():
  spark_koalas_install()
  set_spark_env()
  spark_conf()


In [77]:
# 2.1 
install_spark()


--1.1 Install findspark----


--1.2 Install databricks Koalas----

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

--1.3 Download Apache tar.gz----

--2023-03-24 03:43:24--  https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299360284 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.2-bin-hadoop3.tgz’

spark-3.3.2-bin-had 100%[===================>] 285.49M   132MB/s    in 2.2s    

2023-03-24 03:43:26 (132 MB/s) - ‘spark-3.3.2-bin-hadoop3.tgz’ saved [299360284/299360284]


--1.4 Transfer downloaded content and unzip tar.gz----


--1.5 Check folder for files----

total 292368
drwxr-xr-x  1 root root      4096 Mar 24 03:43 .
drwxr-xr-x  1 root root      4096 Mar 24 01:50 ..
drwxr-xr-x  1 root root     

# B. Call libraries
We do not import `pandas` but `pyspark.pandas` as `ps`.

In [78]:
# 3.0 Just call some libraries to test
import numpy as np
import os
import time 

# 3.1 Get spark in sys.path
import findspark
findspark.init()

# 3.2 Call other spark libraries
#     Just to test
import pyspark.pandas as ps
from pyspark.sql import SparkSession
import databricks.koalas as ks

In [79]:
# 3.3
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [80]:
# 3.4 Increase cell width to display wide columnar output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# C. Build spark session
You can modify spark driver/executor memory here<br>
SparkSession name is `spark`<br>
databricks.koalas is imported as `ks`


## Modifying spark configuraion
Increase driver and executor memory

In [81]:
# 4.0 Check template file
! cat /opt/spark-3.3.2-bin-hadoop3/conf/spark-defaults.conf.template

#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Default system properties included when running spark-submit.
# This is useful for setting default environmental settings.

# Example:
# spark.master                     spark://master:7077
# spark.eventLog.enable

In [82]:
# 4.1 Create spark-defaults.conf 
! cp /opt/spark-3.3.2-bin-hadoop3/conf/spark-defaults.conf.template  /opt/spark-3.3.2-bin-hadoop3/conf/spark-defaults.conf


In [83]:
# 4.2 Amend properties
! echo "spark.driver.memory 6g" >> /opt/spark-3.3.2-bin-hadoop3/conf/spark-defaults.conf
! echo "spark.executor.memory 3g" >> /opt/spark-3.3.2-bin-hadoop3/conf/spark-defaults.conf

In [84]:
# 4.3 Check now
! cat /opt/spark-3.3.2-bin-hadoop3/conf/spark-defaults.conf

#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Default system properties included when running spark-submit.
# This is useful for setting default environmental settings.

# Example:
# spark.master                     spark://master:7077
# spark.eventLog.enable

## Stop and start SparkSession

In [85]:
# 5.0 Build spark session:
# Stop spark, if started
#spark.stop()
# 5.1 Now start spark
spark = SparkSession. \
                    builder. \
                    master("local[*]"). \
                    appName("myexpt"). \
                    getOrCreate()

In [86]:
sc = spark.sparkContext
spark.sparkContext.getConf().getAll()

[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.executor.memory', '3g'),
 ('spark.driver.host', '722e03ecb141'),
 ('spark.app.startTime', '1679622774285'),
 ('spark.app.name', 'myexpt'),
 ('spark.executor.id', 'driver'),
 ('spark.sql.warehouse.dir', 'file:/content/spark-w

In [87]:
# 5.2
print(spark.sparkContext._conf.getAll())

[('spark.driver.extraJavaOptions', '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'), ('spark.executor.memory', '3g'), ('spark.driver.host', '722e03ecb141'), ('spark.app.startTime', '1679622774285'), ('spark.app.name', 'myexpt'), ('spark.executor.id', 'driver'), ('spark.sql.warehouse.dir', 'file:/content/spark-warehouse

# D. Experimentation with pyspark pandas API

### pyspark pandas Series

In [88]:
# 1.0
ps.Series(np.random.random(size = (10,)))

0    0.551771
1    0.828385
2    0.158050
3    0.268507
4    0.891058
5    0.530676
6    0.106150
7    0.416909
8    0.232345
9    0.583707
dtype: float64

In [106]:
# 1.0.1
ps.Series([10,20,30])

0    1
1    2
2    3
dtype: int64

In [107]:
# 1.0.2
ps.Series((4,9,1))

0    4
1    9
2    1
dtype: int64

In [89]:
# 1.0.3
ps.Series(np.random.random(size = (5)), index = ['a', 'b', 'c', 'd', 'e'])

a    0.746913
b    0.133051
c    0.008835
d    0.648483
e    0.301139
dtype: float64

In [90]:
# 1.0.4
ps.Series(np.random.random(size = (5)), index = ['a', 'b', 'c', 'd', 'e']).shape

(5,)

In [ ]:
# 1.0.5
dict(ps.Series(np.random.random(size = (5)), index = ['a', 'b', 'c', 'd', 'e']))

In [92]:
# 1.0.6
# Series to numpy:
# Note the warnings
ps.Series(np.random.random(size = (5)), index = ['a', 'b', 'c', 'd', 'e']).to_numpy()

/opt/spark-3.3.2-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


array([0.65341595, 0.56568903, 0.48566238, 0.77398233, 0.86874306])

In [93]:
# 1.1
abc = ps.Series(np.random.random(size = (5)), index = ['a', 'b', 'c', 'd', 'e'])


In [105]:
# 1.2 Try to press tab after .(dot)
#       and see what all methods are avaiable:

abc.

3.395835994247374

In [99]:
# 1.3
abc.add(abc)

a    1.997965
b    1.413426
c    0.116254
d    1.456330
e    1.807697
dtype: float64

### pyspark pandas DataFrame

In [ ]:
# 2.1
ps.DataFrame([
               [1,2,3],
               [4,5,6]
              ], columns = list('abc') )

,a,b,c
0,1,2,3
1,4,5,6


In [ ]:
# 2.1.1
ps.DataFrame(
              [
                [1,2,3],
                [4,5,6]
              ],
              columns = list('abc'),
              index = [4,7]
             )

,a,b,c
4,1,2,3
7,4,5,6


In [ ]:
# 2.1.2
#     Note how nulls have been inserted.
#     As np.nan and None

pdf = ps.DataFrame(
                   {
                      'x1': ['a','a','b','b',np.nan, 'c', 'd','d'] * 10,
                      'x2': ['apple',np.nan, 'orange','orange', 'peach',np.nan,'apple','orange'] * 10,
                      'x3': [1, 1, 2, 2, 2, 4, 1, 2] *10,
                      'x4': [2.4, None, 3.5, 1.4, 2.1,1.5, 3.0, 2.0] *10,
                      'y1': [1, 0, 1, 0, 0, 1, 1, 0] * 10,
                      'y2': ['yes', 'no', 'no',np.nan,None, 'yes','', 'yes'] *10,
                      'y3': [True, True, False, None, False, True, False,False] *10
                    }
                   )

In [ ]:
# 2.1.3
type(pdf)

print("\n======\n")

# 2.1.4
pdf.head()

In [ ]:
# 2.1.5
pdf.columns

Index(['x1', 'x2', 'x3', 'x4', 'y1', 'y2', 'y3'], dtype='object')

In [ ]:
# 2.1.6
pdf['x1'].head(6)

print("\n======\n")

# 2.1.7
type(pdf['x1'])    # pyspark.pandas.series.Series

0       a
1       a
2       b
3       b
4    None
5       c
Name: x1, dtype: object

pyspark.pandas.series.Series

In [ ]:
# 2.2
pdf[['x1', 'x2']].head()

print("\n======\n")

# 2.3
type(pdf[['x1', 'x2']])   # pyspark.pandas.frame.DataFrame

,x1,x2
0,a,apple
1,a,
2,b,orange
3,b,orange
4,,peach


pyspark.pandas.frame.DataFrame

In [ ]:
# 2.3.1
pdf.shape   # (80,7)

print("\n======\n")

# 2.3.2
pdf.dtypes

(80, 7)

x1     object
x2     object
x3      int64
x4    float64
y1      int64
y2     object
y3     object
dtype: object

In [ ]:
# 2.4 Only numeric features are described:

pdf.describe()

,x3,x4,y1
count,80.000000,70.000000,80.000000
mean,1.875000,2.271429,0.500000
std,0.932874,0.713664,0.503155
min,1.000000,1.400000,0.000000
25%,1.000000,1.500000,0.000000
50%,2.000000,2.100000,0.000000
75%,2.000000,3.000000,1.000000
max,4.000000,3.500000,1.000000


In [ ]:
# 2.5 Describing cat features:

pdf[['x1', 'x2']].describe()

,x1,x2
count,70,60
unique,4,3
top,d,orange
freq,20,30


In [ ]:
# 2.5.1
pdf.info()

<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 80 entries, 0 to 79
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      70 non-null     object 
 1   x2      60 non-null     object 
 2   x3      80 non-null     int64  
 3   x4      70 non-null     float64
 4   y1      80 non-null     int64  
 5   y2      60 non-null     object 
 6   y3      70 non-null     object 
dtypes: float64(1), int64(2), object(4)

In [ ]:
# 3.0 Slicing:

pdf.iloc[:2, :4]

,x1,x2,x3,x4
0,a,apple,1,2.4
1,a,None,1,NaN


In [ ]:
# 3.1 Slicing:

pdf.iloc[[1,3,7], [2,4]]

,x3,y1
1,1,0
3,2,0
7,2,0


In [ ]:
# 3.2 Slicing:

pdf.loc[[1,3,7], ['x1', 'x2']]

,x1,x2
1,a,None
3,b,orange
7,d,orange


### Operations on Series

In [ ]:
# 4.0 Addition
https://stackoverflow.com/a/73701464
ps.Series(np.random.random(size = (10,))) + ps.Series(np.random.random(size = (10,)))

In [115]:
# 4.1 Set option: Default is False to avoid excessive computation:

ps.set_option('compute.ops_on_diff_frames', True)

In [116]:
# 4.2
ps.Series(np.random.random(size = (10,))) + ps.Series(np.random.random(size = (10,)))

0    1.036895
1    0.330829
2    0.493790
3    1.170752
4    0.985252
5    1.285973
6    1.422313
7    0.321337
8    0.828303
9    0.749595
dtype: float64

In [117]:
# 4.3 Add two series with different Index
abc = ps.Series(
                 np.random.normal(loc = 1.0, scale = 2.0, size = (5,) ),
                 index = ['a', 'c', 'b', 'e', 'f']
                )

In [118]:
# 4.3 Add two series with different Index
# 4.3 Add two series with different Index
cde = ps.Series(
                 np.random.normal(loc = 1.0, scale = 2.0, size = (5,) ),
                 index = ['a', 'b', 'c', 'e', 'k']
                )

In [122]:
abc
print("\n========\n")
cde
print("\n========\n")
abc + cde

a   -0.855975
c    3.015098
b   -0.675212
e    6.212318
f    1.581511
dtype: float64

a    1.676420
b   -0.172374
c    3.272379
e   -4.057456
k    6.525404
dtype: float64

a    0.820444
b   -0.847586
c    6.287476
e    2.154862
f         NaN
k         NaN
dtype: float64

### DataFrame operations

#### Filteration

In [124]:
ps.date_range(start = '01/01/2001', end = "01/01/2002", periods = 24)


/opt/spark-3.3.2-bin-hadoop3/python/pyspark/pandas/namespace.py:1881: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  pd.date_range(


DatetimeIndex([       '2001-01-01 00:00:00', '2001-01-16 20:52:10.434782',
               '2001-02-01 17:44:20.869565', '2001-02-17 14:36:31.304347',
               '2001-03-05 11:28:41.739130', '2001-03-21 08:20:52.173913',
               '2001-04-06 05:13:02.608695', '2001-04-22 02:05:13.043478',
               '2001-05-07 22:57:23.478260', '2001-05-23 19:49:33.913043',
               '2001-06-08 16:41:44.347826', '2001-06-24 13:33:54.782608',
               '2001-07-10 10:26:05.217391', '2001-07-26 07:18:15.652173',
               '2001-08-11 04:10:26.086956', '2001-08-27 01:02:36.521739',
               '2001-09-11 21:54:46.956521', '2001-09-27 18:46:57.391304',
               '2001-10-13 15:39:07.826086', '2001-10-29 12:31:18.260869',
               '2001-11-14 09:23:28.695652', '2001-11-30 06:15:39.130434',
               '2001-12-16 03:07:49.565217',        '2002-01-01 00:00:00'],
              dtype='datetime64[ns]', freq=None)

In [126]:
ind = ps.date_range(start = '01-01-2001', end = '01-01-2002', periods = 30)

/opt/spark-3.3.2-bin-hadoop3/python/pyspark/pandas/namespace.py:1881: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  pd.date_range(


In [130]:
ps.DataFrame(np.random.random(size = (30, 5)) , index = ind )

,0,1,2,3,4
0,0.219429,0.897674,0.775683,0.370996,0.785399
1,0.294746,0.924006,0.839199,0.539133,0.916068
2,0.620292,0.136615,0.703763,0.611161,0.707569
3,0.905207,0.781714,0.866988,0.179302,0.865360
4,0.267209,0.793982,0.961784,0.502583,0.961145
5,0.366813,0.172753,0.128472,0.166350,0.275326
6,0.249316,0.234824,0.542417,0.296380,0.483159
7,0.797786,0.125285,0.415713,0.894104,0.486688
8,0.205130,0.562772,0.434515,0.705531,0.226432
9,0.665637,0.395214,0.215116,0.062356,0.702025
